In [3]:
import carla
import time
import pandas as pd

# Connect to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(10.0) 
world = client.get_world()
time.sleep(5)  # ⏳ Allow vehicles to spawn
# Get all vehicles
vehicles = world.get_actors().filter('vehicle.*')

# Initialize data storage
speed_data_base = {v.id: [] for v in vehicles}

# Log speed for 5 minutes (Base Behavior)
start_time = time.time()
sampling_interval = 1.0  # Log speed every 1 second
last_logged_time = start_time

while time.time() - start_time < 60:  
    world.tick()

    
    if time.time() - last_logged_time >= sampling_interval:
        last_logged_time = time.time()
        
        for vehicle in vehicles:
            velocity = vehicle.get_velocity()
            speed = round(3.6 * ((velocity.x**2 + velocity.y**2 + velocity.z**2) ** 0.5), 2)  # Round to 2 decimal places
            speed_data_base[vehicle.id].append(speed)

# Save base behavior data to CSV
df_base = pd.DataFrame.from_dict(speed_data_base, orient='index').transpose()
df_base.to_csv("speed_data_base.csv", index=False)

print("Base behavior speed data saved!")
print(f"📢 Number of vehicles in the simulator: {len(vehicles)}")  # 🔹 Print vehicle count


Base behavior speed data saved!
📢 Number of vehicles in the simulator: 30


In [3]:
import carla
import time
import pandas as pd

# Connect to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
time.sleep(5)  # ⏳ Allow vehicles to spawn

# Get all vehicles
vehicles = world.get_actors().filter('vehicle.*')

# Initialize data storage
tailgating_data = []

# Function to check if agent is tailgating
def is_tailgating(vehicle, world):
    vehicle_list = world.get_actors().filter('vehicle.*')
    for other_vehicle in vehicle_list:
        if other_vehicle.id != vehicle.id:
            distance = vehicle.get_location().distance(other_vehicle.get_location())
            if distance < 10:  # Tailgating if following within 10 meters
                return True, distance
    return False, None

# Log tailgating behavior for 5 minutes
start_time = time.time()
sampling_interval = 3.0  # Log every second
last_logged_time = start_time

while time.time() - start_time < 60:  # 1 minutes
    world.tick()
    
    if time.time() - last_logged_time >= sampling_interval:
        last_logged_time = time.time()
        
        for vehicle in vehicles:
            tailgating, distance = is_tailgating(vehicle, world)
            if tailgating:
                tailgating_data.append({"vehicle_id": vehicle.id, "time": round(time.time() - start_time, 2), "distance": round(distance, 2)})


# Save base tailgating data
df_tailgating = pd.DataFrame(tailgating_data)
df_tailgating.to_csv("tailgating_data_base.csv", index=False)

print("✅ Base tailgating data saved!")


✅ Base tailgating data saved!
